## dev_vmodes_comp
development of vertical mode decomposition for NEMO. z-level with partial step formulation

solving the following problem: $ w' + p = 0, p' = \lambda N^2 w$ with B.C.s $w(-H)=0$ and $w(0)=0$ or free surface. (note that $w$ is not actual vertical velocity)

Free surface reads: $p' + N^2/g p = 0, z=0$. Translates to $w(0)\lambda(1+N^2(0)\delta_z/2) + p(-1/2) = 0$.

Implementation adapted for: 
* grid with negative increasing depth (z(0)=0, z(N)=-H) -> signed reversed for diff matrix
* spacing provided (positive, i.e. with z-cordinate upward )
* staggered formulation with w (N) at left points, p at center. Use w=0 at extra point (bottom) by not writing anything (line substitution)

In [1]:
%matplotlib widget
import matplotlib as mpl
from matplotlib import pyplot as plt

import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as la
from scipy.linalg import eig
from scipy.optimize import root


In [2]:
Nz = 60
Hbot = 2e3
Nsqr = 1e-3 * np.ones(Nz)

free_surf = False
nmodes = 9
sigma = .1
g = 9.81

stretch = "lin" # "exp", "lin"
sig = np.linspace(0, -1, Nz+1)
if stretch == "lin":
    z_f = sig * Hbot
elif stretch == "exp":
    z_f = (np.exp(sig)-1)/(1-np.exp(-1)) * Hbot #sig*Hbot #np.linspace(0, -Hbot, Nz+1)
else:
    raise ValueError('stretching unknon')
    
#z_f = np.linspace()
z_c = .5*(z_f[1:]+z_f[:-1])
if False:
    dzc, dzf = 1./np.arange(1,Nz+1), 1./np.arange(1,Nz)
else:
    dzc, dzf = -np.diff(z_f), -np.diff(z_c)
z_i = z_f[1:-1]

f_test = lambda z: np.exp((z+Hbot)/Hbot) - 1
df_t = lambda z: np.exp((z+Hbot)/Hbot)/Hbot

plt.figure()
plt.plot(np.arange(Nz+1), z_f, ".-")
plt.title("grid")
plt.grid(True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
### vertical derivative matrix, w-to-p grids, taking left w-points (assume w(N+1)=0)
v12 =  np.stack([-1./np.r_[np.ones(1),dzc], 1./np.r_[dzc, np.ones(1)]])
Dw2p = sp.spdiags(v12,[1, 0],Nz,Nz,format="lil")

### vertical derivative matrix, p-to-w grids, targetting inner w points only
v12 =  np.stack([-1./np.r_[np.ones(1),dzf], 1./np.r_[dzf, np.ones(1)]])
Dp2w = sp.spdiags(v12,[1, 0],Nz-1,Nz,format="lil")

fig, axs = plt.subplots(1, 2)
ax = axs[0]
ax.imshow(Dw2p.toarray(), cmap="seismic")
ax = axs[1]
ax.imshow(Dp2w.toarray(), cmap="seismic")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
### check that differentiation matrices are correct
fig, axs = plt.subplots(1, 2, sharey=True)
ax = axs[0]
ax.plot(Dw2p * f_test(z_f[:-1]), z_c, ".-", label="numeric, w2p")
ax.plot(df_t(z_c), z_c, "--", label="analytic, w2p")

ax.plot(Dp2w * f_test(z_c), z_i, marker="+", linestyle="None", label="numeric, p2w")
ax.plot(df_t(z_f), z_f, "--", label="analytic, w2p")
ax.set_title('derivative')
ax.legend()

ax = axs[1]
ax.plot(f_test(z_f), z_f, marker=".", color="k")
ax.set_title(r"function $f(z)$")

for ax in axs:
    ax.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# Adjust matrix for BCs
if Nsqr.size == Nz+1:
    Nsqri = Nsqr[1:-1]
elif Nsqr.size == Nz:
    Nsqri = Nsqr[1:]
    
if free_surf and g>0:
    invg = np.ones(1)/g
else:
    invg = np.zeros(1)
Nsqog = Nsqr[:1]*invg

dz_surf = abs(z_c[0])

### error in the implementation of free surface. 
# I used w=-p', but it is w' = -p and p' = lam*N^2*w. 
# correct, starting from equation p' + N2/g p = 0 at z = 0
B = sp.diags(np.r_[np.zeros(Nz), 1.+Nsqog*dz_surf/2., Nsqri], 0, (2*Nz,2*Nz), format="lil")
Awp = sp.vstack([np.r_[invg, np.zeros(Nz-1)], Dp2w])
Aww = None #sp.diags(np.r_[np.ones(1)+Nsqog*dz_surf, np.zeros(Nz-1)], 0, (Nz,Nz))
A = sp.bmat([ [ sp.identity(Nz), Dw2p ], 
              [ Awp,             Aww  ]
            ])

fig, axs = plt.subplots(1, 2)
ax = axs[0]
ax.imshow(A.toarray())
ax = axs[1]
ax.imshow(B.toarray())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
### analytical solution for free surface

imod = 0

fun = lambda x: x * g / Hbot / Nsqr[0]**.5 * np.sin(x*Nsqr[0]**.5) - np.cos(x*Nsqr[0]**.5)
init_guess = lambda imod: np.pi*imod/Nsqr.mean()**.5
res = root(fun, init_guess(imod))

xx = np.linspace(0,init_guess(imod+2)/Hbot,200)
plt.figure()
plt.plot(xx, fun(xx*Hbot))
plt.plot(res.x/Hbot, 0, "*r")
plt.grid(True)
plt.title("guess: {:.2e}, found: {:.2e}".format((init_guess(imod)/Hbot)**2, (res.x[0]/Hbot)**2))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'guess: 0.00e+00, found: 4.77e-05')

In [7]:
nmod = Nz//4

if free_surf and g>0:
    ana = np.array([root(fun, init_guess(imod)).x[0] for imod in np.arange(nmod)])/Hbot
else:
    ana = init_guess(np.arange(nmod))/Hbot
ana = ana**2

### compute numerical solution
if Nz > 40:
    ev,ef = la.eigs(A.tocsc(), k=nmod, M=B.tocsc(), sigma=sigma)
else:
    Adens = A.toarray()
    Bdens = B.toarray()
    ev, ef = eig(Adens, Bdens)
    
inds = np.isfinite(ev)
ev, ef = ev[inds], ef[:,inds]
pmod, wmod = ef[:Nz,:], -ef[Nz:,:]

isort = np.argsort(ev.real)[:nmod]

fig = plt.figure()
ax = plt.gca()
ax.plot(np.arange(nmod), ev[isort].real, "X")
ax.plot(np.arange(nmod), ana, "+-")
ax.set_ylabel(r'$\lambda$')
bx = ax.twinx()
bx.plot(np.arange(nmod), ev[isort].real - ana, ".-", color="grey")
ax.grid(True)
bx.set_ylabel('error (grey)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'error (grey)')

In [9]:
max_mod = min(5, nmod) # nmod
fig, axs = plt.subplots(2, max_mod, sharey=True, figsize=(max_mod*2, 6))
for imo,ii in enumerate(isort[:max_mod]):
    ax = axs[0,imo]
    ax.plot(pmod[:,ii].real, z_c, pmod[:,ii].imag, z_c, "--")
    ax.plot(Dw2p * wmod[:,ii].real, z_c, "--")
    ax.set_title("{0:.2e} vs {1:.2e}".format(ev[ii].real, ana[imo]))
    ax = axs[1,imo]
    ax.plot(wmod[:,ii].real, z_c, wmod[:,ii].imag, z_c, "--")
for ax in axs.ravel():
    ax.grid(True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
### construct corresponding function
_kwg_vmode = {"free_surf": True, "g": 9.81, Hbot: None, "sigma": .1, "siz_sparse":30} #"nmodes":11, 

def compute_vmodes_1D(N2l, dzc=None, dzf=None, zc=None, zf=None, nmodes=10, **kwargs):
    """
    Compute vertical modes from stratification. 
    Adapted for nemo z-grid with partial steps: ordered with increasing depth, w points = left points
    not tested for other vertical grid types

    Parameters:
    ___________
    N2f: (N) ndarray
        Brunt-Vaisala frequency at cell left faces
    dzc: (N) ndarray, optional
        vertical grid spacing at cell centers
    dzf: (N) ndarray, optional
        vertical grid spacing at cell left points
    zc: (N) ndarray, optional
        vertical grid at cell centers
    zf: (N) or (N+1) ndarray, optional
        vertical grid at cell faces or cell left points
    nmodes: int, optional (default: 10)
        number of baroclinic modes to compute (barotropic mode will be added)
    Hbot: float, optional
        bottom depth. Must be provided if dzc not passed and zf does not include bottom point (size N). Ignored otherwise
    free_surf: bool, optional (default: True)
        whether using a free-surface condition or rigid lid
    g: float, optional (default: 9.81)
        gravity constant (if 0, will do rigid lid, overriding free_surf)
    sigma: float, optional
        parameter for shift-invert method in scipy.linalg.eig (default: _sig)

    Returns:
    ________
    c: (nmodes) ndarray
        eigenvalues (pseudo phase speed, c=1/k)
    phi: (N,nmodes) ndarray
        p-like modes at cell centers
    dphidz: (N+1,nmodes) ndarray
        w-like modes at cell interfaces (= dphi / dz)

    Notes:
    ______
    The vertical modes are definied following the equation:
    .. math:: (\phi'/N^2)' + k^2\phi=0 
    with boundary condition :math:`\phi'=0` at the bottom and :math:`g\phi' + N^2\phi=0` at the surface (or :math:`\phi'=0` for a rigid lid condition). 
    Computation of the vertical modes is performed using second order finite difference on staggered grids

    """ 
    
    ### extract keywords
    kwgs = _kwg_vmode.copy()
    kwgs.update(kwargs)
    free_surf, g = kwgs["free_surf"], kwgs["g"]
    
    ### deal with vertical grids
    Nz = Nsqr.size
    if dzc is not None and dzf is not None:
        if zc is not None:
            dz_surf = abs(zc[0])
        else:
            dz_surf = .25*(dzc[0] + dzf[0]) ### this is NEMO grid
    elif zc is not None and zf is not None:
        if zf.size == zc.size:
            if kwgs["hbot"] is None:
                raise ValueError("must specify w-grid at outer points or Hbot")
            else:
                zf = np.r_[zf, -abs(kwgs["Hbot"])]
        elif zf.size != zc.size+1:
            raise ValueError("incompatible sizes between zc grid and zf grid")
        dzc = np.diff(zf)
        dzf = np.diff(zc)
    else:
        raise ValueError("must specify either grid increments dzc, dzf or z grids zc, zf") 

    ### construct sparse matrices for differentiation
    # vertical derivative matrix, w-to-p grids, taking left w-points (assume w(N+1)=0)
    v12 =  np.stack([-1./np.r_[np.ones(1),dzc], 1./np.r_[dzc, np.ones(1)]])
    Dw2p = sp.spdiags(v12,[1, 0],Nz,Nz,format="lil")

    # vertical derivative matrix, p-to-w grids, targetting inner w points only
    v12 =  np.stack([-1./np.r_[np.ones(1),dzf], 1./np.r_[dzf, np.ones(1)]])
    Dp2w = sp.spdiags(v12,[1, 0],Nz-1,Nz,format="lil")
    
    ### construct eigenproblem. State vector is (p=-w', w) (w is not true vertical velocity)
    # eigen problem is 
    # (1 Dz) * psi = lam * (0 0 ) * psi
    # (Dz 0)               (0 N2)
    # plus corrections for boundary condition at the surface
    if free_surf and g>0:
        invg = np.ones(1)/g
    else:
        invg = np.zeros(1)
    Nsqog = Nsqr[:1]*invg

    B = sp.diags(np.r_[np.zeros(Nz), 1.+Nsqog*dz_surf/2., Nsqr[1:]], 0, (2*Nz,2*Nz), format="lil")
    Awp = sp.vstack([np.r_[-invg, np.zeros(Nz-1)], Dp2w])
    Aww = None #sp.diags(np.r_[np.ones(1)+Nsqog*dz_surf, np.zeros(Nz-1)], 0, (Nz,Nz))
    A = sp.bmat([ [ sp.identity(Nz), Dw2p ], 
                  [ Awp,             Aww  ]
                ])
    
    ### compute numerical solution
    if Nz >= kwgs["siz_sparse"]:
        ev,ef = la.eigs(A.tocsc(), k=nmodes+1, M=B.tocsc(), sigma=sigma)
    else:
        Adens = A.toarray()
        Bdens = B.toarray()
        ev, ef = eig(Adens, Bdens)

    inds = np.isfinite(ev)
    ev, ef = ev[inds].real, ef[:,inds].real
    isort = np.argsort(ev)[:nmodes+1]
    ev, ef = ev[isort], ef[:,isort]
    pmod, wmod = ef[:Nz,:], ev[None,:]*Nsqr[:,None]*ef[Nz:,:]
    return 1./ev**.5, pmod, wmod

def _compute_vmodes_1D_stack(N2l, dzc, dzf, **kwargs):
    return np.vstack(compute_vmodes_1D(N2l, dzc=dzc, dzf=dzf, **kwargs))

In [28]:
c, pp, ww = compute_vmodes_1D(Nsqr, dzc=dzc, dzf=np.r_[dzf[:1], dzf])

In [30]:
res = _compute_vmodes_1D_stack(Nsqr, dzc, np.r_[dzf[:1], dzf], nmodes=11)
res[Nz+1,:]

array([-4.72328368e-08,  8.91826412e-07,  1.82871892e-06,  2.75183983e-06,
        3.66457501e-06, -4.56604410e-06, -5.45436385e-06, -6.32736118e-06,
       -7.18277722e-06,  8.01834306e-06, -8.83181454e-06,  9.62099170e-06])

In [33]:
ww[0,:]

array([-4.72328368e-08, -8.91826412e-07,  1.82871892e-06, -2.75183983e-06,
        3.66457501e-06,  4.56604410e-06, -5.45436385e-06, -6.32736118e-06,
       -7.18277722e-06, -8.01834306e-06, -8.83181454e-06])

In [116]:
imod = 1
plt.figure()
plt.plot(pp[:,imod], z_c)
plt.grid(True)
bx = plt.gca().twiny()
bx.plot(ww[:,imod], z_f[:-1], "--")
bx.plot(np.gradient(pp[:,imod], z_c), z_c, ":")
plt.title("{:.2e}".format(c[imod]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '1.97e+01')

In [120]:
### verify boundary condition
nmod = c.size
dpdz = np.gradient(pp, z_c, axis=0)
dwoNdz = np.gradient(ww/Nsqr[:,None], z_f[:-1], axis=0)
lesig = np.sign(ww[0,:])

plt.figure()
plt.plot(np.arange(nmod), lesig*ww[0,:], "+", 
         np.arange(nmod), -lesig*Nsqr[0]/g*pp[0,:], "--", 
         np.arange(nmod), lesig*Nsqr[0]*c**2/g*dwoNdz[0,:], ":")
plt.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [105]:
ww.shape, Nsqr.shape

((100, 11), (100,))